# **Writeup | Self-Driving Car project - Deep Learning**
**x** min read

**Abstract — This notebook is the writeup of the Traffic Sign Recognition project.** We apply Deep Learning and Convolutional Networks (ConvNets) to the task of traffic sign classification as part of the SELF-DRIVING CAR nanodegree program. The project is broken down into three steps, which are:   
>- Step 1: Data Set Summary and Exploration
- Step 2: Design and Test a Model Architecture
- Step 3: Test a Model on New Images

The model yielded the accuracy of **97.7%** with a loss of **0.141** using 32x32 pre-proceeded input images.

Here is the link to the [PROJECT SPECIFICATION](https://review.udacity.com/#!/rubrics/481/view), here to my [PROJECT CODE](https://github.com/chatmoon/Traffic-Sign-Classifier-Project/blob/master/_2_WIP/_JNBK_/_TSC-step2.1_170309-1557_WIP.ipynb) and here the [Table of Contents](https://github.com/chatmoon/Debugging/blob/master/_WRITEUP_TableOfContents_171104-1145.ipynb) of this Writeup.

---
### Step 1: Data Set Summary & Exploration

The goals here are the following:
* Load the [data set](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset) and Summarize
* Explore and visualize the data set

#### 1.1. Load and summarize the data set  

*The code for this part is between the 1st and 4th cell of the IPython notebook.*   

I used the numpy library to calculate summary statistics of the traffic signs data set:   
* The size of training set is 34799
* The size of validing set is 4410
* The size of test set is 12630
* The shape of a traffic sign image is (32, 32, 3)
* The number of unique classes/labels in the data set is 43

#### 1.2. An exploratory visualization of the dataset
##### 1.2.1. Explore the data set

*The code for this part is between the 5th and 11th cell of the IPython notebook.*   

In this part, we have three representations of the data set:   
- fig.1: a list showing the number of occurence per traffic sign name
- fig.2: a bar chart showing the number of occurence per class id
- fig.3: a bar chart showing the distribution of these traffic sign images into the data set

>![fig.1](https://raw.githubusercontent.com/chatmoon/Traffic_Sign_Classifier/master/images/_dataExplo_1_showList.PNG)
>![fig.2](https://raw.githubusercontent.com/chatmoon/Debugging/master/_images_/_dataExplo_2_showChart.png)
>![fig.3](https://raw.githubusercontent.com/chatmoon/Debugging/master/_images_/_dataExplo_3_showDist.png)

**Observations:**    
- The fig.1 and 2 show there is a large disparity between traffic sign occurences.   
Additional data should be created in order to rebalance the under represented classes.
- The fig.3 shows that each class has been piled on top of the other.   
The data set should be shuffled before training the model.

##### 1.2.2. Visualize the data set
*The code for this part is between the 12th and 16th cell of the IPython notebook.*

In this part, we have also three types of visualization of the data set:   
- fig.4: 43 random images, one per class and with their black-boxes
- fig.5: 5 x 10, 5 random traffic signs, 10 images each 
- fig.6: a sprite image showing all the traffic sign in a single image

> *fig.4: 43 random images, one per class and with their black-box*
![fig.4](https://raw.githubusercontent.com/chatmoon/Traffic_Sign_Classifier/master/images/_dataVisu_4_show43TS.png)
> *fig.5: 5 x 10, 5 random traffic signs, 10 images each*
![fig.5](https://raw.githubusercontent.com/chatmoon/Traffic_Sign_Classifier/master/images/_dataVisu_5_show5TS.png)
> *fig.6: a sprite image showing all the traffic sign in a single image*
![fig.6](https://raw.githubusercontent.com/chatmoon/Traffic_Sign_Classifier/master/images/_sp_xx_5984x5984.png)   
> *fig.7: a sampling of challenging images to classify*
![fig.7](https://raw.githubusercontent.com/chatmoon/Traffic_Sign_Classifier/master/images/_dataExplo_4_challenges.PNG)

**Observations:**   
The fig.4 to 7 show how the images are of different qualities. Many images are either shaky, too dark or not having the same scale. There are variabilities such as viewpoint variations, lighting conditions (saturations, low-contrast), motion-blur, occlusions, sun glare, physical damage, colors fading. The classification of the traffic sign can be challenging and complex in this context. We should pre-proceed the images to decrease the impact of the mixed qualities of images.

**Note:** the sprite image will be useful for the embedding visualization in TensorBoard.

---
### Step 2: Design and Test a Model Architecture

#### 2.1. Preprocess the data set and generate additional data

##### 2.1.1. Preprocess the data set
*The code for this part is between the 18th and 21st cell of the IPython notebook, and 29th and 30rd cell.*

> **Definition:** Pre-processing refers to techniques such as converting to grayscale, normalization, etc.

In this part, I describe how I preprocessed the image data, what techniques were chosen and why I chose these techniques. You will also find below a overview of the preprocessing workflow in figure *fig.8* with images showing the output of each preprocessing technique.

In summary, the preprocessing workflow generate five different types of image: 0RGB, 1GRAY, 2SHP, 3HST, 4CLAHE.

> *fig.8: the preprocessing workflow*
![fig.8](https://raw.githubusercontent.com/chatmoon/Traffic_Sign_Classifier/master/images/_dataPPro%20flow_170610-1221.png)


First of all, **all preprocessed images are centered and normalized** because during the training of the network we multiply weights to the initial input and add biases to cause activations and then backpropagate with the gradients to train (update) the model. In this process, we do not want the gradients go out of control. Then all preprocessed images are centered around zero by subtracting the mean, and normalized by dividing by the standard deviation. This technique doesn't change the content of the image. It avoids the values of weights and biases to get too big or to small. It tackles the numerical stability issue that occurs when several small values are added to big values (introducing a lot of errors) during the optimization of the Loss function.

The two starting points of this approch are:
- the following comments: "the ConvNet was trained with full supervision on the colorimages of the GTSRB dataset and reached 98.97% accuracyon the phase 1 test set. After the end of phase 1, additional experiments with grayscale images established a new record accuracy of 99.17%", Traffic Sign Recognition with Multi-Scale Convolutional Networks, Pierre Sermanet and Yann LeCun
- the observation of the training set samples shows there are variabilities such as colors fading, lighting conditions (saturations, low-contrast), sun glare, motion-blur. To overcome a part of these variabilities and make the classification easier, I converted RGB images to grascale, then grayscale images have been sharpened, the histograms of sharpened images have been equilized, and the histograms of equilized images have been equilized adaptively with limited contrast

The following functions from OpenCV have been used:
- 1GRAY: [cvtColor](http://docs.opencv.org/2.4/modules/imgproc/doc/miscellaneous_transformations.html#cvtcolor)   
- 2SHP: [filter2D](http://docs.opencv.org/2.4/modules/imgproc/doc/filtering.html#filter2d)   
- 3HST: [equalizeHist](http://docs.opencv.org/2.4/modules/imgproc/doc/histograms.html#equalizehist) to improve the contrast   
- 4CLAHE: [createCLAHE](http://docs.opencv.org/3.1.0/d5/daf/tutorial_py_histogram_equalization.html)   

**Note:**
- In addition, I also tried to blur the images but I got a better accuracy removing this last technic   
- other realistic perturbations would probably also increase robustness of the model such as other affine transformations, brightness or adding some artificial occlusions. They would be implemented in the future sprint

##### 2.1.2. Generate additional data
*The code for this part is between the 22nd and 28th cell of the IPython notebook, and 31st and 35th cell.*

In this part, I describe how and why I generated additional data, what techniques were chosen and why I chose these techniques. You will also find below a workflow of the generation of additional data in figure fig.10 with a visualization of the jittered image in fig.9.

I decided to generate additional data for two reasons:
- first, the fig.2 shows the data set is imbalanced, i.e. the classes are not represented equally. In this case, the accuracy measures might be excellent accuracy on paper but it is only reflecting the underlying class distribution. For example, if the accuracy is 90% of the instances in Class-3 (Speed limit 50km/h) is because the models look at the data and cleverly decide that the best thing to do is to always predict “Class-3” and achieve high accuracy
- secondly, the amount of data might be not sufficient for the model to generalise well in production with new data   


To add more data to the the data set, I combined the two following techniques:
- images are randomly picked and perturbed in position ([-2,2] pixels)
- then they are pertubed in rotation ([-15,+15] degrees)   


**Notes:**
- in addition, I also tried to use the bounding box to crop the images and then perturbing them in scale ([.9,1.1] ratio). I got an accuracy around 93%, far below the human performance of 98.81%. I removed this last part and I get a better result at the end      
- the observation of the training set in fig.6 shows the data set is a stack of several series of 30 similar images **with usually increasing scale**. It is for that reason I did not implement the simple perturbation in scale
- there are 21 traffic signs that have a horizontal or a vertical axis of symmetry. Consequently, they are invariant to horizontal or vertical flipping. This technic would be implemented to add more data to the data set in the future sprint

Here is an example of an original image and an augmented image:
> *fig.9: augmented data example*
![fig.9](https://raw.githubusercontent.com/chatmoon/Traffic_Sign_Classifier/master/images/_dataJit_test4.png)

I did not know how much the data would have to be raised to improve the accuracy or to overcome the overfitting problem. Then I created several set of augmented data such as each class has a least the following quantity (qty) of occurence: 500, 1000, 1500, 2000, 2500 and 3000 for each preprocessed type of image (see fig.10).
> *fig.10: workflow of the generation of additional data*
![fig.10](https://raw.githubusercontent.com/chatmoon/Traffic_Sign_Classifier/master/images/_dataJIT_flow_170617-2336.png)   
> *tab.11: the size of augmented training set*

| Minimum quantity per class | 0 | 500 | 1000 | 1500 | 2000 | 2500 | 3000 |
|:---------------------------------------------------------------:| 
| Quantity in addion | 0 | 4440 | 12451 | 15690 | 18630 | 21490 | 21500 |
| Total size | 34799 | 39239 | 51690 | 67380 | 86010 | 107500 | 129000 |

#### 2.2. Model Architecture   
*The code for this part is between the 36th and 44th cell of the IPython notebook.*   

In this part, I describe what the final model architecture used looks like, how the model has been trained and the approach taken for finding a solution.

##### 2.2.1. The final model architecture

*The code for this part is in the 44th cell of the IPython notebook.*   

Even though I have built and experimented [several models](https://raw.githubusercontent.com/chatmoon/Traffic_Sign_Classifier/master/images/_architecture_170711-1428.png), I get the best test and validation accuracies with the Lenet5 architecture.

> *fig.12: the original LeNet5 architecture*
![fig.12](https://raw.githubusercontent.com/chatmoon/Traffic_Sign_Classifier/master/images/lenet5.png)

Here is a diagram describing the final model:
> *fig.13: the final model architecture used*
![fig.13](https://raw.githubusercontent.com/chatmoon/Traffic_Sign_Classifier/master/images/_architecture_170711-1428_mod1.png)

The model consisted of the following layers:   

| Layer         		|     Description	        					 | 
|:---------------------:|:----------------------------------------------:| 
| Input         		| 32x32xch, ch=3 for RGB image or 1 for grayscale| 
| Convolution        	| 1x1 stride, valid padding, outputs 28x28x6   	 |
| RELU					|												 |
| Max pooling	      	| 2x2 stride,  outputs 14x14x6  				 |
| Convolution   	    | 1x1 stride, valid padding, outputs 10x10x16	 |
| RELU					|												 |
| Max pooling	      	| 2x2 stride,  outputs 5x5x16  				     |
| Flatten	      		| outputs 400  				    			     |
| Fully connected		| outputs 200  				    			     |
| RELU					|												 |
| Dropout				| keep_prob = 0.67								 |
| Fully connected		| outputs 84  				    			     |
| RELU					|												 |
| Dropout				| keep_prob = 0.67								 |
| Fully connected		| outputs 43  				    			     |


**Note:** The graph of the model can be found using Tensorboard: [graph](https://raw.githubusercontent.com/chatmoon/Traffic_Sign_Classifier/master/images/_architecture_tensorboard.png) 


##### 2.2.2. How the model has been trained

*The code for this part is between the 46th and 51st cell of the IPython notebook.*

In this part, the discussion includes the type of optimizer, the batch size, number of epochs and any hyperparameters such as learning rate.

To train the model, I used the Adam optimyzer. Adam stands for Adaptive moment estimation. It usually outperforms the other types of optimizer like the gradient descent, the stochastic gradient descent, the mini-batch gradient descent, the momentum or the Nesterov accelerated gradient. For more details, see this great short [video](https://www.youtube.com/watch?v=nhqo0u1a6fw).   

I used ReLU as activation function for the hidden layers. I implemented and tested leaky ReLU but the ReLU function gave me the best validation and test accuracies results. It outperforms the other activation functions like Sigmoid and hyperbolic tangent function (tanh). I did not try Maxout function. For more details, see this short [video](https://www.youtube.com/watch?v=-7scQpJT7uo).

I used the dropout technique as regularization method with a rate of 0.67.

Regarding the other hyperparameters:   

| Hyperparameter    	| Value  |   
|:---------------------:|:------:|   
| LEARNING RATE       	| 8.5e-4 |   
| EPOCHS   	            | 100	 |   
| BATCH SIZE            | 100	 |

##### 2.2.3. The approach taken for finding a solution

###### 2.2.3.0. Description of the general approach

It was an iterative approach.   

As a prelude, I used the Lenet5 architecture with the initial RGB images as input data and arbitrary hyperparameter values.   
I chose this starting point for two reasons:   
- Firstly, the initial measure gave me a benchmark value of the validation accuracy that would serve as an element of comparaison when optimizing the architecture performance. At this stage, I got a validation accuracy of 91.7%, a test accuracy of 90.3% and a loss of 0.456 with a learning rate of 1E-3.  
- Secondly, the Lenet5 architecture is a building block of the Multi-Scale Convolutional Network. ![ MultiScaleConvNet](https://ai2-s2-public.s3.amazonaws.com/figures/2016-11-08/9ab0de951cc9cdf16887b1f841f8da6affc9c0de/1-Figure2-1.png)   
The latter promises a validation accuracy of 99.17%, above the human performance of 98.81%. In this way, I was able to get a sense of the impact of each parameter on a simpler system before playing with more complex architectures.   

Next, I played with the following parameters to tune the Lenet5 model:   
- learning rate: [1E-3, 2E-3, 9E-4, 1E-4, 1E-5, 8E-4, 9.5E-4, 8.5E-4]
- dropout rate: [0.5, 0.75, 0.25, 0.85, 0.6, 0.8, 0.67]
- preprocessed data types: (centered & normalized) AND [RGB, grayscale, sharpen, histogram, CLAHE]
- the amount of jittered data: [500, 1000, 1500, 2000, 2500, 3000]   
- activation function: [Relu, leakyRelu]

As a result, I got a better performance: a validation accuracy of 97.7%, a test accuracy of 95.8% with a loss of 0.141.

Finally, I played with various changes in the initial architecture and tried other ones.   

###### 2.2.3.1. Initial benchmark values with the Lenet5 architecture

*The code for this part is between the 52nd and 55th cell of the IPython notebook.*

At this stage, I got a validation accuracy of 91.7%, a test accuracy of 90.3% and a loss of 0.456 with a learning rate of 1E-3.   

The fig.14.a shows the model does not overfit or underfit.     

> *fig.14.a: model1 - cost and accuracies measures before, in between and tuning*
![fig.14.a](https://raw.githubusercontent.com/chatmoon/Traffic_Sign_Classifier/4521055bf8aaa1f74f97d2eb83f0853ac51567f3/images/_TensorBoard_Mod1_variousLearningRate.png)


###### 2.2.3.2. Tuning of the Lenet5 architecture

By varying the learning rate, I was able to get a validation accuracy of 94.4% with a learning rate of 8.5E-4 *(see fig. 15.a)*.   

A learning rate of 8.5E-4 would be kept for the next measures.

> *fig.15.a: model1's cost and accuracies measures with various learning rates*
![fig.15.a]( https://raw.githubusercontent.com/chatmoon/Traffic_Sign_Classifier/master/images/_scatterMod1_variousLearningRate.png)
*Find [here](https://github.com/chatmoon/Traffic_Sign_Classifier/blob/master/jpbk/_SimulationResults_171029-1039.ipynb) the details of the model1's cost and accuracies measure.*   

By varying the dropout rate, I got better performance: a validation accuracy of 94.7% and a test accuracy of 93.5% with a dropout rate of 0.67 *(see fig. 15.b)*.   

> *fig.15.b: model1's cost and accuracies measures with various dropout rates*
![fig.15.b]( https://raw.githubusercontent.com/chatmoon/Traffic_Sign_Classifier/master/images/_scatterMod1_variousDropoutRates.png)
*Find [here](https://github.com/chatmoon/Traffic_Sign_Classifier/blob/master/jpbk/_SimulationResults_171029-1039.ipynb) the details of the model1's cost and accuracies measure.*   

By varying the preprocessed data types in input, I got an improvment using centered and normalized data, and with grayscale and RGB images *(see fig. 16)*:
- grayscale & centered & normalized: validation accuracy of 96.3%, a test accuracy of 94.5%, a cost of 0.279   
- RGB & centered & normalized: validation accuracy of 94.9%, a test accuracy of 94.7%, a cost of 0.492   

> *fig.16: model1's cost and accuracies measures with various with various preprocessed data types*
![fig.16](https://raw.githubusercontent.com/chatmoon/Traffic_Sign_Classifier/master/images/_scatterMod1_variousPproDataType.png)
*Find [here](https://github.com/chatmoon/Traffic_Sign_Classifier/blob/master/jpbk/_SimulationResults_171029-1039.ipynb) the details of the model1's cost and accuracies measure.*   

By varying the amount of jittered data in input, I got an improvment using at least 3000 centered and normalized RGB images per class: a validation accuracy of 97.7%, a test accuracy of 95.8%, a cost of 0.141 *(see fig. 17)*.   

> *fig.17: model1's cost and accuracies measures with various with various amount of jittered grayscale & RGB data*
![fig.17](https://raw.githubusercontent.com/chatmoon/Traffic_Sign_Classifier/master/images/_scatterMod1_variousJitData.png)
*Find [here](https://github.com/chatmoon/Traffic_Sign_Classifier/blob/master/jpbk/_SimulationResults_171029-1039.ipynb) the details of the model1's cost and accuracies measure.*   

###### 2.2.3.3. Other architectures

Finally, by varying architectures, the Lenet5 architecture has still gotten the best performance results *(see fig.18, .19)*.   

> *fig.18: cost and accuracies measures with various architectures*
![fig.18](https://raw.githubusercontent.com/chatmoon/Traffic_Sign_Classifier/master/images/_scatterModXNew_result.png)
*Find [here](https://raw.githubusercontent.com/chatmoon/Traffic_Sign_Classifier/master/images/_architecture_170711-1428.png) the details of each model.*   

> *fig.19: different architectures descriptiion that have been tested*
![fig.19](https://raw.githubusercontent.com/chatmoon/Traffic_Sign_Classifier/master/images/_architecture_170711-1429.PNG)
*Find [here](https://raw.githubusercontent.com/chatmoon/Traffic_Sign_Classifier/master/images/_architecture_170711-1428.png) the details of each model.*  

###### 2.2.3.4. In conclusion

I must say that I am a bit desappointed. Even though I implemented the Multi-Scale Convolutional Network and used various pre-processing data techniques, the performance was not as good as the record of 99.17% promised by the paper. I expected to improve the performance by playing with various architectures but I was not successful despite my several attempts and due to some hardware limitions. Only the standardization, normalization and jitteration techniques have a significant impact on the validation and test accuracies. 

In conclusion, after building and experimenting seven models, the best validation and test accuracies have been achieved with the Lenet5 architecture, using 32x32x3, jittered, centered and normalized RGB images. **The model yielded a validation accuracy of 97.7%, a test accuracy of 95.8% with a loss of 0.141.**   

---
### Step 3: Test a Model on New Images

In this part, I tested the model with new images. These latter were taken on German streets or they were screenshots of street view from Google Map.

If you look at fig.4, you will find there are five general shapes that describe the 43 traffic signs. These genral shapes are shown in the fig.20 below:
> *fig.20: five general shapes describing the 43 traffic signs*
![fig.20](https://raw.githubusercontent.com/chatmoon/Traffic_Sign_Classifier/master/images/_Step3_/_TS_5_general_shapes.PNG)

Then I created three series of images:
- serie 01: it contents only five traffic signs with shape 1, cercle
- serie 02: it contents only five traffic signs with shape 2, triangle pointing upward
- serie 03: it contents five traffic signs with only one shape from any shape


#### 3.1. Load and Output the Images

*The code for this part is between the 56th and 59th cell of the IPython notebook.*

The fig.21.1 shows the five German traffic signs of serie 01
- the first line shows the original images
- the second line shows RGB images that were centered and normalized
- the thrid line shows grayscale images that were also centered and normalized
> *fig.21.1: serie 1 - five new German traffic signs*
![fig.21.1](https://raw.githubusercontent.com/chatmoon/Traffic_Sign_Classifier/master/images/_Step3_/_serie01_/_ownImages_serie01_0_ori-rgb-gray.PNG)


The fig.21.2 shows the five German traffic signs of serie 02
> *fig.21.2: serie 2 - five new German traffic signs*
![fig.21.2](https://raw.githubusercontent.com/chatmoon/Traffic_Sign_Classifier/master/images/_Step3_/_serie02_/_ownImages_serie02_0_ori-rgb-gray.PNG)


The fig.21.3 shows the five German traffic signs of serie 03
> *fig.21.3: serie 3 - five new German traffic signs*
![fig.21.3](https://raw.githubusercontent.com/chatmoon/Traffic_Sign_Classifier/master/images/_Step3_/_serie03_/_ownImages_serie03_0_ori-rgb-gray.PNG)


**Observations:**   
The images are of good qualities. They have similar viewpoint orientation. They are not shaky, too dark or having different scale. They don't have major imperfections such as motion-blur, occlusions (except a tiny occlusion on the first image), sun glare or physical damage. But there are variabilities such as lighting conditions (saturations, low-contrast) and about the background contents.




#### 3.2. The model's predictions on these new traffic signs   

*The code for this part is between the 61st and 66th cell of the IPython notebook.*

The model was able to correctly guess **0 of the 5 traffic signs**, which gives **an accuracy of 0%**.   

The fig.22 shows the predictions on the serie01 images in grayscale.
> *fig.22: serie 1 - predictions on the grayscale images*
![fig.22](https://raw.githubusercontent.com/chatmoon/Traffic_Sign_Classifier/master/images/_Step3_/_serie01_/_ownImages_serie01_gray_0_prediction.PNG)   

| Image			        |     Prediction    | 
|:---------------------:|:-----------------:| 
| Speed limit (20km)	| Double curve      | 
| Speed limit (30km)	| Speed limit (70km)|
| Speed limit (50km)	| Speed limit (70km)|
| End of all speed   	| Pedestrians       | 
| Ahead only			| Speed limit (70km)|

The model recognized none of them. Furthermore, five circle shapes have been given as an input. The prediction managed to recognized two triangles. How is it possible?

Having come a long way, I don't get it why the model fails to generalize with these images. I am on the very verge of throwing my laptop through the window and then myself... Fortunately I live on the first floor.    

Having said that, I tried to run the prediction on images from the training, validation and test data set. And again, the model yielded a validation accuracy of 0%. Why that? How could the model yield a test accuracy of 95% and then it fails to recognize images from the training, validation and test data set?   

In addition, the prediction results are not repeatable. Each time I run prediction with the same input images, I get different results.

What can I do to analyze the code and to find out the problem?   

#### 3.3. The top 5 softmax probabilities and certainty of the model's prediction  

*The code for this part is between the 67th and 73th cell of the IPython notebook.*

In this part, we describe how certain the model is when predicting on each of the new images by looking at the softmax probabilities for each prediction.

The fig.23.1 shows the top five predictions on the serie01 images in grayscale.   
The fig.23.2 shows the visualization of predictions as bar chart.   
And the fig.23.3 replaces the bar chat by an image with the prediction as a title.   

> *fig.23.1: serie 1 - predictions on the grayscale images*
![fig.23.1](https://raw.githubusercontent.com/chatmoon/Traffic_Sign_Classifier/master/images/_Step3_/_serie01_/_ownImages_serie01_gray_3_topK_image.png)   
> *fig.23.2: serie 1 - visualization of predictions as bar chart*    
![fig.23.2](https://raw.githubusercontent.com/chatmoon/Debugging/master/_images_/_Step3_/_serie01_/_ownImages_serie01_gray_2_topK43_chartBar.png)   
> *fig.23.3: serie 1 - visualization of predictions*    
![fig.23.3](https://raw.githubusercontent.com/chatmoon/Debugging/master/_images_/_Step3_/_serie01_/_ownImages_serie01_gray_3_topK35_image.png)

The fig.23.1 shows the top five predictions for each images vary between 3% to 5%. How could the top predictions be so low? Namely, for the three speed limits images, the model hesitates between one circle and four triangles. Is the prediction methode or the data processing or the training process that should be analyzed to solve this issue? 

The fig.23.2 and fig.23.3 shows a visualization of the predictions.   
For the first image, the right anwser is ranked in the 13th position.   
For the second image, it is ranked in the second postion.   
For the third image, it is ranked in the 11th position.   
For the fourth image, it is ranked in the 35th position.   
Finally, for the fifth image, it is ranked in the 34th position.  

# ANNEX

[writeup_template.md](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md)